In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
log_file_path = "C:/Users/nickgy/Documents/Haltech/Data Logs/20171014.csv"

In [3]:
# Read csv log file
df=pd.read_csv(log_file_path, skiprows=63, header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,06:03:31.000,458.0,300.0,0.0,0.0,4.0,3111.0,3309.0,950.0,1166.0,312.0,12793.0,116.0,1.0,312.0,4.0,312.0,51.0,165.0,0.0
1,06:03:31.062,460.0,NaN,NaN,NaN,NaN,NaN,NaN,948.0,1161.0,314.0,12793.0,116.0,-1.0,313.0,3.0,314.0,52.0,NaN,NaN
2,06:03:31.078,460.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,06:03:31.187,NaN,NaN,NaN,NaN,NaN,NaN,NaN,950.0,1170.0,313.0,12778.0,117.0,-1.0,313.0,3.0,313.0,50.0,NaN,NaN
4,06:03:31.203,458.0,NaN,NaN,NaN,4.0,NaN,NaN,947.0,1159.0,312.0,12793.0,NaN,1.0,312.0,4.0,312.0,54.0,NaN,NaN


In [4]:
# Rename headers
headers = ["Time", "BaseFuel", "BaseIgnition", "FuelCoolantTempCorrection", "FuelAirTempCorrection", "ThrottlePosition", "AirTemp", 
          "CoolantTemp", "LambdaSensor1", "RPM", "Load", "BatteryVoltage", "EngineRunningTime", "DeltaLoad", "MAPSource",
          "TransientThrottleLoadSource", "IgnitionLoad", "AFRDifference", "IgnitionTiming", "TransientThrottleDLoadDisp"]
df.columns = headers

In [5]:
# Set time as index
df.set_index("Time", inplace=True)
df.index

Index(['06:03:31.000', '06:03:31.062', '06:03:31.078', '06:03:31.187',
       '06:03:31.203', '06:03:31.296', '06:03:31.312', '06:03:31.328',
       '06:03:31.359', '06:03:31.375',
       ...
       '06:26:55.031', '06:26:55.046', '06:26:55.078', '06:26:55.093',
       '06:26:55.109', '06:26:55.125', '06:26:55.156', '06:26:55.203',
       '06:26:55.234', '06:26:55.234'],
      dtype='object', name='Time', length=42106)

In [6]:
# Convert time index type from object to DatetimeIndex
df.index = df.index.to_datetime()
df.index

c:\program files\python35\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: to_datetime is deprecated. Use pd.to_datetime(...)
  


DatetimeIndex([       '2017-10-15 06:03:31', '2017-10-15 06:03:31.062000',
               '2017-10-15 06:03:31.078000', '2017-10-15 06:03:31.187000',
               '2017-10-15 06:03:31.203000', '2017-10-15 06:03:31.296000',
               '2017-10-15 06:03:31.312000', '2017-10-15 06:03:31.328000',
               '2017-10-15 06:03:31.359000', '2017-10-15 06:03:31.375000',
               ...
               '2017-10-15 06:26:55.031000', '2017-10-15 06:26:55.046000',
               '2017-10-15 06:26:55.078000', '2017-10-15 06:26:55.093000',
               '2017-10-15 06:26:55.109000', '2017-10-15 06:26:55.125000',
               '2017-10-15 06:26:55.156000', '2017-10-15 06:26:55.203000',
               '2017-10-15 06:26:55.234000', '2017-10-15 06:26:55.234000'],
              dtype='datetime64[ns]', length=42106, freq=None)

In [7]:
# Convert raw values to presentation values

def convert_pressure_kpa_abs_to_kpa(val):
    return val - 101.3

def convert_afr_lambda_to_afr(val):
    return val * 14.7

def map_afr_diff(raw_val):
    return convert_afr_lambda_to_afr(raw_val / 1000)

def map_load(raw_val):
    return convert_pressure_kpa_abs_to_kpa(raw_val / 10)

def convert_temp_kelvin_to_celsius(val):
    return val - 273.15

def map_temp(raw_val):
    return convert_temp_kelvin_to_celsius(raw_val / 10)
    
    
df['AFRDifference'] = df['AFRDifference'].apply(map_afr_diff);

df['Load'] = df['Load'].apply(map_load);
df['MAPSource'] = df['MAPSource'].apply(map_load);
df['IgnitionLoad'] = df['IgnitionLoad'].apply(map_load);

df['AirTemp'] = df['AirTemp'].apply(map_temp);
df['CoolantTemp'] = df['CoolantTemp'].apply(map_temp);

In [8]:
df.describe()

,BaseFuel,BaseIgnition,FuelCoolantTempCorrection,FuelAirTempCorrection,ThrottlePosition,AirTemp,CoolantTemp,LambdaSensor1,RPM,Load,BatteryVoltage,EngineRunningTime,DeltaLoad,MAPSource,TransientThrottleLoadSource,IgnitionLoad,AFRDifference,IgnitionTiming,TransientThrottleDLoadDisp
count,36361.000000,22820.000000,3.0,3.0,18654.000000,13640.000000,14472.000000,38899.000000,40451.000000,37292.000000,38123.000000,2232.000000,22665.000000,37377.000000,18606.000000,37293.000000,39397.000000,23212.000000,3.0
mean,515.951817,370.861174,0.0,0.0,39.705908,55.730821,76.913039,981.582689,1662.298855,-60.185831,12579.103638,389.106183,-0.008074,-60.195374,39.974309,-60.184206,0.185254,192.184603,0.0
std,99.200112,61.949872,0.0,0.0,43.451800,9.287044,8.327118,83.265286,553.454293,15.806522,174.997847,274.249905,1.317085,15.795104,43.399357,15.806959,1.254105,64.561175,0.0
min,330.000000,200.000000,0.0,0.0,3.000000,37.950000,57.750000,713.000000,0.000000,-86.400000,9603.000000,0.000000,-31.000000,-86.400000,-1.000000,-86.400000,-4.983300,40.000000,0.0
25%,434.000000,310.000000,0.0,0.0,7.000000,47.950000,70.650000,928.000000,1248.000000,-72.300000,12434.000000,141.000000,0.000000,-72.300000,7.000000,-72.300000,-0.646800,142.000000,0.0
50%,507.000000,366.000000,0.0,0.0,24.000000,58.650000,75.950000,975.000000,1414.000000,-65.000000,12606.000000,350.000000,0.000000,-65.000000,25.000000,-65.000000,0.102900,182.000000,0.0
75%,596.000000,423.000000,0.0,0.0,62.000000,63.950000,85.850000,1050.000000,1968.000000,-50.100000,12715.000000,629.250000,1.000000,-50.200000,62.000000,-50.100000,0.882000,246.000000,0.0
max,794.000000,533.000000,0.0,0.0,299.000000,66.850000,88.750000,1277.000000,4249.000000,-2.400000,13043.000000,908.000000,22.000000,-2.400000,299.000000,-2.400000,5.027400,361.000000,0.0


In [9]:
df[["AFRDifference", "RPM", "Load"]]

,AFRDifference,RPM,Load
2017-10-15 06:03:31.000,0.7497,1166.0,-70.1
2017-10-15 06:03:31.062,0.7644,1161.0,-69.9
2017-10-15 06:03:31.078,NaN,NaN,NaN
2017-10-15 06:03:31.187,0.7350,1170.0,-70.0
2017-10-15 06:03:31.203,0.7938,1159.0,-70.1
2017-10-15 06:03:31.296,0.7350,1165.0,-70.0
2017-10-15 06:03:31.312,0.8085,1166.0,-70.2
2017-10-15 06:03:31.328,0.8085,1167.0,-70.0
2017-10-15 06:03:31.359,0.8232,1174.0,NaN
2017-10-15 06:03:31.375,0.8379,NaN,-70.0


In [10]:
# Group by 500 RPM ranges and 10 kPa load ranges; then aggregate
dfgrouped=df.groupby([pd.cut(df["RPM"], np.arange(0, 10500, 500)), pd.cut(df["Load"], np.arange(-100, 10, 10))]).agg(["mean", "count"])

In [11]:
dfgrouped["AFRDifference"].dropna(axis=0).sort_values(ascending=False, by=["mean", "count"])

mean  count
RPM          Load                       
(3500, 4000] (-90, -80]  3.586800      7
(3000, 3500] (-90, -80]  3.046470    140
(2500, 3000] (-90, -80]  2.640870    149
(4000, 4500] (-80, -70]  2.352000      1
(1500, 2000] (-90, -80]  2.116714    171
(3000, 3500] (-80, -70]  2.086133     58
(2500, 3000] (-70, -60]  1.934100     63
             (-80, -70]  1.825895     95
(4000, 4500] (-60, -50]  1.499400      1
(3000, 3500] (-70, -60]  1.489174     23
(4000, 4500] (-50, -40]  1.484700      2
(2000, 2500] (-90, -80]  1.399789    523
(3500, 4000] (-80, -70]  1.190700      1
(3000, 3500] (-50, -40]  1.134656     16
(3500, 4000] (-70, -60]  1.102500      2
             (-60, -50]  1.058400      1
             (-50, -40]  1.036350      2
(1500, 2000] (-80, -70]  0.970722   2730
(2500, 3000] (-50, -40]  0.912707    135
(2000, 2500] (-80, -70]  0.818216    755
(1000, 1500] (-70, -60]  0.711900   6074
(2500, 3000] (-40, -30]  0.698506    344
(2000, 2500] (-20, -10]  0.592535     94
(4000, 4500] (-30, -20]  0.543900      1
(3000, 3500] (-40, -30]  0.499561    123
(500, 1000]  (-60, -50]  0.458373    121
(2500, 3000] (-60, -50]  0.455329    317
(2000, 2500] (-40, -30]  0.380216    652
(3500, 4000] (-40, -30]  0.376047     43
(2000, 2500] (-30, -20]  0.347846    273
(1000, 1500] (-60, -50]  0.321068   1431
(500, 1000]  (-50, -40]  0.284690     30
(2000, 2500] (-70, -60]  0.243517    304
(1500, 2000] (-70, -60]  0.184947    946
(3500, 4000] (-20, -10]  0.163878     54
(2000, 2500] (-60, -50]  0.109836   1528
(4000, 4500] (-20, -10]  0.016660     30
(1000, 1500] (-30, -20]  0.013808    379
(3000, 3500] (-20, -10] -0.033531    121
(500, 1000]  (-40, -30] -0.062954     46
(3000, 3500] (-30, -20] -0.079094    431
(500, 1000]  (-20, -10] -0.088200      2
(2000, 2500] (-50, -40] -0.094593   1060
(500, 1000]  (-70, -60] -0.149138     55
(2500, 3000] (-30, -20] -0.225256    272
(3000, 3500] (-60, -50] -0.251580    210
(1500, 2000] (-50, -40] -0.256727   1841
(1000, 1500] (-50, -40] -0.265618    953
(2500, 3000] (-20, -10] -0.367733     63
(1000, 1500] (-80, -70] -0.395117   8931
(1500, 2000] (-60, -50] -0.486042   1389
             (-40, -30] -0.494734   1033
(1000, 1500] (-40, -30] -0.536108    532
(2000, 2500] (-10, 0]   -0.635559     17
(1500, 2000] (-30, -20] -0.700259    289
(500, 1000]  (-30, -20] -0.914073     11
(1500, 2000] (-20, -10] -1.160075     48
             (-10, 0]   -2.113860      5